In [1]:
import pandas as pd
import sys
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
import pickle



Using TensorFlow backend.


In [9]:
with open('./output/preprocess_object.pickle','rb') as f:
    preprocess_object = pickle.load(f)

dataset = pd.read_csv('./output/train_dataset.csv',delimiter=',',encoding='utf-8',low_memory=False)

number_df = dataset.select_dtypes(include=['int64','float64'])
#스케일러를 통한 전체 자료 분포 평균0, 분산1로 변환(오버플로우, 언더플로우 방지, 독립변수 공분산 행렬 조건수 감소) 
#--> 안정성 및 수렴속도 향상
scaler = preprocess_object.pop('scaler')

scaler_arr = scaler.transform(number_df)
train_data = pd.DataFrame(scaler_arr, columns=number_df.columns)

categorical = list(dataset.select_dtypes(include=['object']).columns)
oh_encoder ={}

for cat in categorical:
    oh_encoder = preprocess_object[cat]
    le = oh_encoder['LabelEncoder']
    ohe = oh_encoder['OneHotEncoder']
    cat_le_arr =le.transform(dataset[cat])
    cat_ohe_arr = ohe.transform(cat_le_arr.reshape(-1,1)).toarray()
    oh_df = pd.DataFrame(cat_ohe_arr, columns=[cat+'_'+le.inverse_transform([int(i)])[0] for i in range(cat_ohe_arr.shape[1])])
    train_data=pd.concat([train_data, oh_df], axis=1)




In [10]:
print(train_data['resp_N'])
train_label = pd.DataFrame()
train_label['resp_N'] = train_data['resp_N']
train_label['resp_Y'] = train_data['resp_Y']

del train_data['resp_N']
del train_data['resp_Y']


0        0.0
1        0.0
2        0.0
3        1.0
4        0.0
5        1.0
6        1.0
7        1.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       1.0
15       1.0
16       1.0
17       1.0
18       0.0
19       1.0
20       0.0
21       0.0
22       1.0
23       1.0
24       1.0
25       0.0
26       0.0
27       0.0
28       0.0
29       1.0
        ... 
25618    1.0
25619    1.0
25620    0.0
25621    0.0
25622    1.0
25623    1.0
25624    0.0
25625    0.0
25626    0.0
25627    1.0
25628    0.0
25629    1.0
25630    1.0
25631    0.0
25632    0.0
25633    0.0
25634    0.0
25635    0.0
25636    0.0
25637    0.0
25638    0.0
25639    0.0
25640    0.0
25641    0.0
25642    0.0
25643    0.0
25644    0.0
25645    0.0
25646    1.0
25647    0.0
Name: resp_N, Length: 25648, dtype: float64


In [12]:
model = Sequential()
model.add(Dense(train_data.shape[1],input_dim=train_data.shape[1], activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [13]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_data,train_label,epochs=10, batch_size=100)

Epoch 1/10
25648/25648 [==============================] - 5s 189us/step - loss: 0.5696 - acc: 0.7079
Epoch 2/10
25648/25648 [==============================] - 2s 75us/step - loss: 0.5447 - acc: 0.7239
Epoch 3/10
25648/25648 [==============================] - 2s 76us/step - loss: 0.5299 - acc: 0.7334
Epoch 4/10
25648/25648 [==============================] - 2s 76us/step - loss: 0.5138 - acc: 0.7430
Epoch 5/10
25648/25648 [==============================] - 2s 76us/step - loss: 0.4950 - acc: 0.7568
Epoch 6/10
25648/25648 [==============================] - 2s 76us/step - loss: 0.4745 - acc: 0.7660
Epoch 7/10
25648/25648 [==============================] - 2s 77us/step - loss: 0.4487 - acc: 0.7815
Epoch 8/10
25648/25648 [==============================] - 2s 77us/step - loss: 0.4216 - acc: 0.7967
Epoch 9/10
25648/25648 [==============================] - 2s 77us/step - loss: 0.3954 - acc: 0.8104
Epoch 10/10
25648/25648 [==============================] - 2s 77us/step - loss: 0.3675 - acc: 0.827

In [15]:
save_file='train_keras_model.h5'
model.save('./output/'+save_file)

print("\n Accuracy: %.4f" % (model.evaluate(train_data, train_label)[1]))

25648/25648 [==============================] - 1s 42us/step

 Accuracy: 0.8541
